In [6]:
import os
from getpass import getpass

# Mount Google Drive
if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Google Drive already mounted")

# Git configuration
!git config --global user.name "sammig6i"
!git config --global user.email "swghazzawi@gmail.com"

# Securely input your GitHub personal access token
token = getpass('Enter your GitHub token: ')

# Clone the GitHub repository
!git clone https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git

# Copy the notebook to the repository
!cp /content/drive/MyDrive/NBA_Fantasy_Clustering.ipynb /content/NBA-Fantasy-Points-Prediction/notebooks/

# Navigate to the repository directory
%cd /content/NBA-Fantasy-Points-Prediction

# Add the notebook file to the repository
!git add notebooks/NBA_Fantasy_Clustering.ipynb

# Commit the changes
!git commit -m "Add clustering notebook for NBA Fantasy project"

# Push the changes to GitHub securely
!git push https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git


Google Drive already mounted
/content/NBA-Fantasy-Points-Prediction
[main 448eb64] Add clustering notebook for NBA Fantasy project
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/NBA_Fantasy_Clustering.ipynb (94%)
fatal: could not read Username for 'https://github.com': No such device or address
